<br>
<h1><center> <br> "From GD to Ranger"</center></h1>

<h2><center>Cutting-Edge Gradient Descent Variants in Theory and Practice</center></h2>

<br>
<h3><center>Levente Szabados & Miklós Tóth</center></h3>




<center><h1>What is gradient descent?</h1></center>

<center><img src="https://miro.medium.com/max/1200/1*iNPHcCxIvcm7RwkRaMTx1g.jpeg" width=45%></center>

<center><img src="https://miro.medium.com/max/1446/1*7VyTVnlVj2Ooqa3MRSRFEQ.gif" width=75%>    

<center><img src="https://www.sciencemag.org/sites/default/files/styles/inline__450w__no_aspect/public/ma_0504_NID_alchemy_WEB.jpg?itok=OzWhcdrK" width=45%></center>

<center><h4>In reality, there are way more dimensions, hence it is MUCH more compliacted.</h4></center>

## Basic forms of gradient descent
### "Vanilla" == "Batch" == "Full" == "Full batch" Gradient Descent
Calculates the full gradient for the loss on the **whole training data** set and updates the weights with it $\Rightarrow$ **update is only once per epoch**.
#### Properties
<ul style='list-style-type: none;'>
<li><b>+</b> Updates with the true, full gradient for the whole dataset
<li><b>+</b> The individual gradients can be computed in parallel -- useful for concurrent implementations
<li><b>+</b> The full gradient updates can lead to less noise in convergence
<li><b>-</b> For large data sets doing only one update per epoch leads to very slow convergence
<li><b>-</b> Requires accumulating a huge amount of gradients for larger data sets can be computationally intensive
<li><b>-</b> Works with the full training gradient without any type of regularization.
</ul>

### Stochastic Gradient Descent
<br></br>
**Approximates** the full gradient by calculating the gradient for a **single example**. **Updates weights with the computed gradient after each example.** The "stochastic" label comes from the fact that (in contrast to full GD) the (typically random) order of the examples influences the trajectory of the parameter evolution. 

#### Properties
<ul style='list-style-type: none;'>
<li><b>+</b> The frequent updates can lead to very fast learning in certain cases
<li><b>+ Noisy update process can help avoiding local minima</b>
<li><b>+ Approximating the gradient is a type of regularization</b>
<li><b>-</b> The very frequent updates are computationally expensive
<li><b>-</b> Cannot be parallelized
<li><b>- The noisy update process can make the convergence difficult</b> 
</ul>

### Minibatch Stochastic Gradient Descent

A balanced middle ground between full and stochastic gradient descent: it divides the training set into **smaller sets**, the so called **"minibatches"**. **For each minibatch**, the gradient for the data points in the minibatch is calculated and **an update is made**. Minibatch size varies by application -- it is typically between 32 and 256.

<ul style='list-style-type: none;'>
<li><b>+</b> Relatively frequent updates help fast convergence
<li><b>+</b> Using an approximation of the full gradient has a regularizing effect and helps avoiding local minima
<li><b>+</b> Can be parallelized, utilizes fast, hardware supported tensor operations
<li><b>-</b> Introduces an additional hyperparameter, the minibatch size
<li><b>-</b> Requires the summation of individual gradients in the minibatch
</ul>

<font color='red'>Important:</font> Somewhat confusingly, in the AI/ML community by SGD people commonly mean Minibatch Stochastic Gradient Descent... 

A nice introduction to the GD variants, on which the present discussion was based, can be found here: [A Gentle Introduction to Mini-Batch Gradient Descent and How to Configure Batch Size](https://machinelearningmastery.com/gentle-introduction-mini-batch-gradient-descent-configure-batch-size)

See also [Ruder's discussion](http://ruder.io/optimizing-gradient-descent).

# More than a tradeoff

<center><img src="https://miro.medium.com/max/1816/1*3L2t1Da4M3ztbB0I1Torhw.png" width=55%></center>

The ability to **navigate complex loss terrains** is one of the key features of non-full GD methods.

Moreover: According to [some research eg. by Vidal et al.](https://www.youtube.com/watch?v=eEPXTMHnBJA) non-full gradient descent, that is **"gradient noise" is oe of the keys for generalization** in deep networks.

## GD optimization algorithms

### Problems/Challenges for vanilla GD

- It is difficult to find the proper learning rate. With a wrong value the loss can be divergent or convergence can be very slow.
- Learning rate regimes are general and rigid, the rate is not adapted to the data set.
- It might be advisable to update different parameters with different learning rates, e.g. we might want to have larger updates for examples with very rare features.
- The problem of **saddle points**, where the gradients are 0 but it is not a local extremum, e.g. because the point is a local minimum in one dimension and a local maximum in another:

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1e/Saddle_point.svg/300px-Saddle_point.svg.png" width=45%></center>

- In general, vanilla minibatch GD does not make use of the earlier gradients, that is, it ignores  information about the curve that is contained in the past trajectory.

The most well known improved GD variants in use are the following:

## Idea 1: Keeping the momentum


### Momentum

This method calculates the new update vector at time $t$, $\mathbf v_t$  by adding the **weighted previous $\mathbf v_{t-1}$ update vector** to the actual gradient:

<br></br>

<center><img src="https://ae01.alicdn.com/kf/Hd3f6c7ff6d5946979d6be6e01de5336ew.gif" width=45%></center>

#### Mathematically:

\begin{align}
\mathbf v_t &= \gamma \mathbf v_{t-1} + \eta \nabla_\theta J( \theta_t) \\  
\theta_{t+1} &= \theta_t - \mathbf v_t
\end{align}

Where the $\gamma$ weight is typically set to a value close to but smaller than 1, e.g. 0.9, and $\eta$ is the learning rate.

#### Consequences:

- Momentum can **move over saddle points** because the inherited momentum term will can change the parameters even if the local gradient is zero.
- Compared to vanilla (S)GD the update component increases for dimensions whose gradient's sign remains the same across updates while decreases for directions whose gradient's sign oscillates. The consequences are reduced oscillation and faster convergence.

<center>
Without momentum:</center>


<center>
<img src="http://ruder.io/content/images/2015/12/without_momentum.gif" width=40%></center>


<center>
With momentum:</center>

<center>
<img src="http://ruder.io/content/images/2015/12/with_momentum.gif" width=40%>

<center>
(Both figures from <a href="http://ruder.io/optimizing-gradient-descent/">Ruder's GD optimization blog entry</a>)
</center>

### Nesterov momentum (NAG)

A small modification of momentum, which tries to "look ahead" when calculating the gradient. Since momentum adds the momentum term $\gamma \mathbf v_{t-1}$ to the computed gradient at $\theta_t$, we  can use this term to look ahead and calculate the gradient for a point which is a bit further in the direction we are heading,  namely, at $\theta_t -  \gamma \mathbf v_{t-1}$. 

<center><img src="http://cs231n.github.io/assets/nn3/nesterov.jpeg">

(The source of the figure is <a href="http://cs231n.github.io/neural-networks-3/">Karpathy's discussion</a>.)

The equations with this modification are 

\begin{align}
\mathbf v_t &= \gamma \mathbf v_{t-1} + \eta \nabla_\theta J( \theta_t-\gamma v_{t-1}) \\  
\theta_{t+1} &= \theta_t - \mathbf v_t
\end{align}

## Idea 2: Variable learning rate

### Adagrad

In contrast to the previous methods, this is not concerned with the direction of the update vectors, but with **adapting the learning rate for the individual parameters based on their past gradients**.

Concretely, Adagrad maintains the $s^p$ running sum of squares of the previous subgradients for each $p$ parameter (weight or bias), and uses it to adapt the (separate) learning rate for the parameters at each step.

#### Mathematically

$$
p_{t+1} = p_{t} - \frac{\eta}{\sqrt {s^p_{t} + \epsilon}} \Delta_pJ(\theta_t)
$$

$$
s^p_{t}=s^p_{t-1} + (\Delta_pJ(\theta_{t}))^2
$$

$\epsilon$ here is a small (typically $\sim 10^{-8}$) smoothing term to avoid division by zero.

Using the $\odot$ elementwise (Hadamard) product operation between vectors we can write the update rules in a vectorized form:

$$
\theta_{t+1} = \theta_{t} - \frac{\eta}{\sqrt {\mathbf s_{t} + \epsilon}} \odot \Delta_pJ(\theta_t)
$$

$$
\mathbf s_{t}=\mathbf s_{t-1} + \Delta_\theta J(\theta_t)\odot \Delta_\theta J(\theta_t)
$$

where all scalar operations on $\mathbf s$ in $\frac{\eta}{\sqrt {\mathbf s_{t} + \epsilon}}$ in the first equation are to be interpreted elementwise.

#### Technically


<center><img src="https://image.slidesharecdn.com/anoverviewofgradientdescentoptimizationalgorithms-170414055411/95/an-overview-of-gradient-descent-optimization-algorithms-47-638.jpg?cb=1492149859" width=55%>

**Advantages:**
- The different learning rates are very useful for convergence, especially since Adagrad can selectively make larger updates to weights that are associated with sparse data (many 0 subgradients), while smaller ones to weights for more frequent features.
- Requires no learning rate tuning/regime -- $\eta$ is typically set to the default (0.01) at the beginning and remains unchanged.

**Problems:**
- The **running gradient sum of squares are constantly growing** so the learning rates monotonically decrease and become very small  basically stopping the learning process.

### RMSProp

Tries to solve the main problem of Adagrad: the aggressive, monotonic decrease of the adaptive learning rates.  The RMSprop solution is to apply a _decay_ term $\rho$ in the maintained running sum of the squared subgradients, so the new value is

$$
\mathbf s_{t} = \rho \mathbf s_{t-1} + (1-\rho) (\Delta_\theta J(\theta_t)\odot\Delta_\theta J(\theta_t))
$$
the result is that the effect of older subgradienst is deminishing over time. ($\rho$ is typically set to 0.9, like the momentum term).



<center><img src="http://drive.google.com/uc?export=view&id=1Gybb0BsWBHEdXS6xZXo7-obB3F7L-Tuy" width=75%>

## (First) combination

What if we put momentum and adaptive learning rates together?


### Adam 

Adam (Adaptive Moment Estimation) is similar to RMSprop in that it adapts the learning rates for parameters by maintaining running sums of the squares of subgradients but -- somewhat similarly to momentum -- it also relies on the previous gradient vectors in the form of a decayed sum:

$$
\mathbf s_{t} = \beta_1 \mathbf {s}_{t-1} + (1-\beta_1) \Delta_\theta J(\theta_t)\odot\Delta_\theta J(\theta_t)
$$

$$ \mathbf m_{t} = \beta_2 \mathbf m_{t-1} + (1-\beta_2)\Delta_\theta J(\theta_t)$$

Since with 0 initialization and the usually slow decay ($\beta$ and $\delta$ are very close to 0) these sums tend to be biased towards $\mathbf 0$ so the update rule uses unbiased values

$$ \hat{\mathbf s}_t = \frac{\mathbf s_t}{1-\beta_1} $$
$$ \hat{\mathbf m}_t = \frac{\mathbf m_t}{1-\beta_2} $$

With these unbiased values, the update rule is a combination of the Momentum and the Adagrad rule:


$$
\theta_{t+1} = \theta_{t} - \frac{\eta}{\sqrt {\hat{\mathbf s_{t}} + \epsilon}} \odot \hat{\mathbf m_t}
$$

The recommended settings for $\beta_1$ and $\beta_2$ are 0.9  for $\beta_1$,  0.999 for $\beta_2$, and $10^{-8}$ for ϵ.

It worked pretty well, so for a time, Adam was the "king of the jungle".

<center><img src="https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/05/Comparison-of-Adam-to-Other-Optimization-Algorithms-Training-a-Multilayer-Perceptron.png" width=45%>

### Adadelta

Similarly to Adam, Adadelta can also be considered a combination of RMSprop and Momentum. In addition to accumulating the usual $\mathbf s$ running sum of the squared gradients, Adadelta also keeps track of a $\mathbf v$ decayed sum of previous parameter updates:

$$\mathbf s_{t} = \rho \mathbf s_{t-1} + (1-\rho) \Delta_\theta J(\theta_t)\odot\Delta_\theta J(\theta_t)
$$

$$\mathbf v_{t} = \rho\mathbf v_{t-1}  + (1-\rho)(\theta_{t+1}-\theta_t)\odot(\theta_{t+1}-\theta_t)  $$

The Adadelta update rule using these running sums is 

$$
\theta_{t+1} = \theta_{t} - \frac{\sqrt {\mathbf v_{t-1} + \epsilon}}{\sqrt {\mathbf s_{t} + \epsilon}} \odot \Delta_\theta J(\theta_t)
$$

($\mathbf v _{t-1}$ is used because we don't know yet $\mathbf v _{t}$ at the update).

Notice that in contrast to all methods we heve seen, here the $\eta$ learning rate has been completely eliminated.  The only parameter to be set is the decay term $\rho$ which is typically set to a value close to 1, say 0.9. 




### Demonstration

(source: [Optimizing gradient descent](http://ruder.io/optimizing-gradient-descent/index.html))

<center><img src="https://cdn-images-1.medium.com/max/800/1*XVFmo9NxLnwDr3SxzKy-rA.gif" width=45%></center>

<center><img src="https://d33ypg4xwx0n86.cloudfront.net/direct?url=https%3A%2F%2Fcdn-images-1.medium.com%2Fmax%2F1600%2F1*SjtKOauOXFVjWRR7iCtHiA.gif&resize=w704" width=45%></center>

### Recommended readings about GD optimization algorithms

- [Ruder:  An overview of gradient descent optimization algorithms](http://ruder.io/optimizing-gradient-descent/)
- [Karpathy: Stanford cs231n  part 3](http://cs231n.github.io/neural-networks-3/)

### Tensorflow implementations

- [Momentum](https://www.tensorflow.org/api_docs/python/tf/train/MomentumOptimizer)
- [Adagrad](https://www.tensorflow.org/api_docs/python/tf/train/AdagradOptimizer)
- [Adam](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer)
- [AdamW](https://www.tensorflow.org/api_docs/python/tf/contrib/opt/AdamWOptimizer)
- [RMSprop](https://www.tensorflow.org/api_docs/python/tf/train/RMSPropOptimizer)
- [Adadelta](https://www.tensorflow.org/api_docs/python/tf/train/AdadeltaOptimizer)


### Sidenote: AdamW
 
An interesting development for Adam is that after the initial hype it was not used much because researchers found it's generalization ability to be worse than the alternatives (e.g. simple Momentum). Now it seems that the some problems were inherent in the complicated interaction between Adam and and regularization, specifically L2 weight decay -- in contrast to vanilla SGD there is a huge difference between adding L2 to the loss function vs applying weight decay during updates. Basically: **One had to patch Adam, and do it the right way.** And the latter, called **AdamW**  seems to lead to way better results. See the blog entry [AdamW and Super-convergence is now the fastest way to train neural nets](http://www.fast.ai/2018/07/02/adam-weight-decay/) for details.

But still, **some of the performance issues remained.** - see below.

### Problems with adaptive learning rate methods

As the paper [Improving Generalization Performance by Switching from Adam to SGD](https://arxiv.org/abs/1712.07628) pointed out, adaptive learning rate methods many times **fail to generalize as strongly as SGD**. so in practice there is a trade-off between the speed improvements of the adaptive methods and the quality of the solution.

<center><img src="http://drive.google.com/uc?export=view&id=1f-g_MBVr6wws7PR5rS_CinHHcLndPqDg" width=35%>

This leads to two conclusions:

1. **Be cautious with using adaptive methods**
2. There is more research needed in this direction. In fact, new state-of-the-art optimization methods as [RAdam](https://arxiv.org/abs/1908.03265v1) or [AdaBound](https://medium.com/syncedreview/iclr-2019-fast-as-adam-good-as-sgd-new-optimizer-has-both-78e37e8f9a34) try to explicitly deal with this issue.

# Parallel development:

## Learning rate and it's "schedules"


Although, as we have already seen, there are GD variants that adaptively set the learning rate during training (Adadelta etc.), **non-adaptive GD variants**, e.g. vanilla SGD and Momentum are still frequently used for optimization and require **manual setting**.

For these variants, after setting an **initial learning rate** (which is a crucial hyperparameter), the rate is typically kept constant during an epoch, but is changed (typically decreased) after every epoch (or every $n$ epochs with a fixed $n$) **according to a schedule**. 

The most frequently used schedules are:

+ **Step decay**: The learning rate is reduced by a $\rho$ factor after every epoch or $n$ epochs, that is an
$$\text{lr} = \varrho \cdot \text{lr}$$ 
update is performed where $0<\varrho<1$. 

+ **Exponential decay**: The learning rate is set to be
$$\text{lr} = \text{lr}_0\cdot e^{-k t}
$$
where $k$ is a hyperparameter and $t$ is the number of already trained epochs (or $n$ epochs). It is easy to see that this is equivalent to a step decay with $\varrho = e^{-k}$.
+ **${1}/{t}$ decay**: The learning rate is set to
$$\text{lr} = \frac{\text{lr}_0 }{1 + k t}$$ where $k$ is a hyperparameter and $t$ is the number of already trained epochs (or $n$ epochs).
+ **Constant learning rate** across epochs.

These schedules can be combined into more complex ones, e.g. a schedule may keep the learning rate constant for a number of epochs and then start a step decay. The switch between the simple schedules can happen simply at a predetermined epoch, but it is frequently connected to a validation metric, e.g. can be triggered when a metric has stopped improving.

<center><h3>Step decay

<center><img src="https://cdn-images-1.medium.com/max/1600/1*VQkTnjr2VJOz0R2m4hDucQ.jpeg" width=60%>


<center><h3>Exponential decay

<center><img src="https://cdn-images-1.medium.com/max/1200/1*iSZv0xuVCsCCK7Z4UiXf2g.jpeg" width=60%>


<center><a href="https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1">source</a>

## Only decreasing?

**If it is true, that the loss topology is "bumpy", does it mske sense to ONLY decrease the LR?**

What if we sometimes **when we are stuck, we increase a bit, and then decrease again**?

What if we anticipate this in advance?

<center><img src="http://drive.google.com/uc?export=view&id=1wyitNljanUgywx3NuKaMv9FqLpyw8oVL" width=55%>

<center><img src="http://drive.google.com/uc?export=view&id=1xwLNwnfIYMsdXl_lgtRI_4FBNJs1-wYI" width=55%>

## Warm restarts

Original paper:
["Stochastic gradient descent with warm restarts"](https://arxiv.org/abs/1608.03983) 

"In this paper, we propose to periodically simulate warm restarts of SGD, where in each restart the
learning rate is initialized to some value and is scheduled to decrease. Four different instantiations
of this new learning rate schedule are visualized in Figure 1. Our empirical results suggest that SGD
with warm restarts requires 2× to 4× fewer epochs than the currently-used learning rate schedule
schemes to achieve comparable or even better results."

<img src="http://drive.google.com/uc?export=view&id=12a2yIch8Nnf27g8UEQ9ydJy9MhuAzysR" width=65%>

## Sidenote: "Superconvergence"

The usage of cyclic LR has some interesting sideeffects also, which made it very popular. Enter **"superconvergence"**.

"we describe a phenomenon, which we named “super-convergence”, where neural networks can be trained an order of magnitude faster than with standard training methods. The existence of super-convergence is relevant to understanding why deep networks generalize well. One of the key elements of super-convergence is training with one learning rate cycle and a large maximum learning rate. A primary insight that allows super-convergence training is that large learning rates regularize the training, hence requiring a reduction of all other forms of regularization in order to preserve an optimal regularization balance."

<center><img src="https://cdn-images-1.medium.com/max/1200/1*cmYfSsmlXm8XdjNddsQqZw.jpeg" width=65%>

<br></br>

<center>Original source <a href="https://arxiv.org/pdf/1708.07120.pdf">here</a>

<center>More approachable summary <a href="https://towardsdatascience.com/https-medium-com-super-convergence-very-fast-training-of-neural-networks-using-large-learning-rates-decb689b9eb0">here</a>

### The "why" for cyclic LR:

"The motivation for the “One Cycle” policy was the following: The learning rate **initially starts small to allow convergence to begin but as the network traverses the flat valley, the learning rate becomes large, allowing for faster progress through the valley**. In the final stages of the training, when the training needs to settle into the local minimum, the learning rate is once again reduced to a small value."

<img src="https://cdn-images-1.medium.com/max/1200/1*Y3Xnw8qxOH6zdGmTlCMDbA.jpeg" width=65%>

### Snapshot ensembles

Warm restarts represented originally a separate line of research, but got combined with ["Snapshot ensembles"](https://arxiv.org/abs/1704.00109). This technique holds a connection - even in name - with ensemble models. 

During this procedure, we **save all "good" model states - typically before warm restarts - and finally do an ensemble** of them in the classical sense.

It's biggest (proposed) advantage is, that we hope to collect solutions from similarly powerful, but distinct optima, which were visited during training, so their ensemble will be more powerful than any of them alone.

<center><img src="http://ruder.io/content/images/2017/11/snapshot_ensembles.png" width=75%>

This also gives an answer to the question - posed in early stopping context - if we should store and use the final model. No, not necessarily.

Moreover if we accept the fact that a number of "global" optima are present for a given model, we are inclined to **"sample" from more "basins"**, which even connects us to boosting methods.

For opinions on this [this](http://www.argmin.net/2018/02/05/linearization/) post is worth reading. It can point towards a more unified view.

An also very interesting summary of the developments in this regard is [this](http://ruder.io/deep-learning-optimization-2017/).


(Sebastian and the team at AYLIEN are very up to date in NLP, worth following [here](http://ruder.io/#open).)

# What does this have to do with new optimizers?

## An unexpected side-effect of warmup: Better optimizers!

### "Rectified Adam" - RAdam

The "one cycle" policy, especially it's first part, the **"warmup" phase**, where we **start with a small initial learning rate, and gradually increase it** can have unexpected sideeffects in case of adaptive learning rate optimizers, especially **Adam**, since **warmup mitigates the generalization problem of adaptive LR methods**.

The crucial understanding of the paper ["On the Variance of the Adaptive Learning Rate and Beyond"](https://arxiv.org/abs/1908.03265) is, that much of the loss in generalization performance in case of Adam comes from it's naive over reliance on the **initial variance in gradients**, thus the **weight distribution gets quickly distorted** and never fully finds it's way back to fruitful, more global optima.

<center><img src="http://drive.google.com/uc?export=view&id=1mYfms1HJeL7O_OuvPcDg3HZpdVpyh2ow" width=45%>

So the newest, state of the art optimization method seems to be a form of Adam, namely **rectified Adam (or RAdam)** that incorporates some learnings from the warm-up method and variable learning rates.

_It is basically trying to set up an adaptive regularization scheme with which it balances the amount of Adam's adaptive LR properties, thus in extreme case it can act as SGD, disregarding the aggregated variance data, and only if appropriate does it start to behave like Adam._

Or with the words of the authors:

"Comparing these two strategies (warmup and RAdam), RAdam deactivates the adaptive learning rate when its variance is divergent, thus avoiding undesired instability in the first few updates."
 
A nice introduction can be found [here](https://medium.com/@lessw/new-state-of-the-art-ai-optimizer-rectified-adam-radam-5d854730807b).

It promises to be fast, but more importantly **robust across a wide selection of learning rates**. 

<img src="https://miro.medium.com/max/700/1*BMwu8Km-CtPsvaH8OM5_-g.jpeg" width=65%>

Since the [paper](https://arxiv.org/abs/1908.03265) describing the method is still pretty fresh, the verdict is still out, but looks very promising! (Implementations are not yet totally mainstream...)

### Lookahead

Though the inspiration is not that direct and obvious, but the idea of storing some weights during the training had some "spin-off" ideas in optimization. In their recent paper [LookAhead optimizer: k steps forward, 1 step back](https://arxiv.org/abs/1907.08610) Zhang et al. proposed an optimization method where they **keep a copy of the weights, and use two optimization regimes, one "slow" and one "fast" for the network.**

<center><img src="http://drive.google.com/uc?export=view&id=1EhpRMvpgKowimKMnuVrkcXtd6hB79iIo" width=70%></center>
    
<br></br>

<center><b>After a short period (some some iterations, eg. 5) they than "synchronize" the weights.</b></center>

Or with the words of the authors: 

"Lookahead maintains a set of slow weights $φ$ and fast weights $θ$, which get synced with the fast weights every $k$ updates. The fast weights are updated through applying $A$, any standard optimization algorithm, to batches of training examples sampled from the dataset $D$. After $k$ inner optimizer updates using $A$, the slow weights are updated towards the fast weights by linearly interpolating in weight space, $θ − φ$. We denote the slow weights learning rate as $α$. After each slow weights update, the fast weights are reset to the current slow weights value."

#### Why is this good?

As this [excellent description](https://medium.com/@lessw/new-deep-learning-optimizer-ranger-synergistic-combination-of-radam-lookahead-for-the-best-of-2dc83f79a48d) puts it:

"...in effect **it allows a faster set of weights to ‘look ahead’ or explore** while **the slower weights stay behind to provide longer term stability**. The result is reduced variance during training, and **much less sensitivity to sub-optimal hyper-parameters** and reduces the need for extensive hyper-parameter tuning... By way of simple analogy, LookAhead can be thought of as the following. Imagine you are at the top of a mountain range, with various dropoff’s all around. One of them leads to the bottom and success, but others are simply crevasses with no good ending. To explore by yourself would be hard because you’d have to drop down each one, and assuming it was a dead end, find your way back out. But, if you had a buddy who would stay at or near the top and help pull you back up if things didn’t look good, you’d probably make a lot more progress towards finding the best way down because **exploring the full terrain would proceed much more quickly and with far less likelihood of being stuck in a bad crevasse.**"

<center><img src="http://drive.google.com/uc?export=view&id=1A43MSBp0s-zKO8H8EtUxY_B1rcTLJnRC" width=85%></center> 
<br></br>
<br></br>

<center><b>The thing seems to actually work!</b></center>

### Surprise, surprise: A combination, Ranger!

Well, if both RAdam and Lookahead achieved impressive new state-of-the-art results, why not combine the two?

This is exactly what happened, thus the new optimization method [Ranger](https://medium.com/@lessw/new-deep-learning-optimizer-ranger-synergistic-combination-of-radam-lookahead-for-the-best-of-2dc83f79a48d) was born.

"**The result is in effect a fast moving average from the internal optimizer (in this case RAdam) and a slower exponential moving average via LookAhead.** The fast one explores but the slow one serves as the pull-back or stability mechanism — normally staying behind while the faster average explores, but in some cases pushing the faster one down a more promising slope while the faster one continues to explore.
By virtue of having the safety of LookAhead, the optimizer can more fully explore the landscape without as much concern over becoming stuck.
**This approach is completely different than the two main methods currently in use — adaptive momentum or ‘heavy ball’ /Nesterov type momentum.**
Thus, LookAhead end’s up out-exploring and finding ‘the way down’ faster and more robustly due to enhancing training stability, and thus outperforming even SGD."

<center><img src="http://drive.google.com/uc?export=view&id=1MhBVj3Kdfo_1ZhNDQ-PnIW6e9-BICq9P" width=45%>

**The results were impressive, the normalization effects of RAdam at the beginning, and the overall stabilization of Lookahead combine seamlessly!**

As of 2019 September, this was the state-of-the-art, but the saga continues...

# Epilogue: How can you use it?

```python
!pip install tensorflow-addons

radam = tfa.optimizers.RectifiedAdam()
ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)

...

model.compile(optimizer=ranger, ...)```
